In [ ]:
import pandas as pd 
from urllib import request
import json
import time

In [ ]:
url = 'http://192.168.0.124/php/diagnosis.php?getrangingdiagnosis=421000000000'
tag = '1231'
url = url + tag
columns = ['Ranging', 'Duration']
recv_freq = 0.5

In [ ]:
def recv_data(url, drop_columns, start_time):
    data = request.urlopen(url).read().decode("utf-8")
    cur_time = time.time()
    locator_data = []
    data_str = json.loads(data)
    for idx in range(0, len(data_str)):
        locator = list(data_str.keys())[idx]
        locator_data.append(json.loads(data_str[locator]))
    locator_frame = pd.DataFrame(locator_data, index=list(data_str.keys()))
    #locator_frame[['Rx','Ry','Rz','Ax','Ay','Az']] = locator_frame['IMU'].str.split(',', expand=True)
    locator_frame['Duration'] = cur_time-start_time
    locator_frame = locator_frame[columns]
    return locator_frame, cur_time

In [ ]:
data_cnt = 1
start_time = time.time()
result = pd.DataFrame()
while(1):
    try:
        try:
            cur_data, cur_time = recv_data(url, columns, start_time)
            result = result.append(cur_data)    
            print("資料筆數:",data_cnt,", 持續時間:",cur_time-start_time)
            data_cnt = data_cnt + 1
        except:
            print('沒有數據!')
        time.sleep(recv_freq)
    except KeyboardInterrupt:
        break

In [ ]:
csv_name = 'uwb' + '.csv'
pd.to_csv(csv_name, index=False)